In [1]:
#!pip install gcsfs

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import glob
import os
#import gcsfs
#import tensorflow as tf
#from tensorflow.python.lib.io import file_io

In [ ]:
path =r'C:/case files/fh_20190217/full_history'
#fs = gcsfs.GCSFileSystem(project='Case1-TTPRSD')
allFiles = glob.glob(path + "/*.csv")
#allFiles = fs.ls('stockdatagrp21/full_history')
# allFiles
alldf = []
i=0

for file_ in allFiles:
    i+=1
    if i%500 == 0:
        print(i)
    if i==2000:
        break
#    with fs.open(file_) as f:
    dftemp = pd.read_csv(file_,index_col=None, header=0)
#     dftemp = pd.read_csv(file_,index_col=None, header=0)
#     dftemp = df_npd.compute()
    #Figure out a way to invert the data frame
    dftemp = dftemp[::-1]
    #Calculating the 200 day moving average before merging all data
    dftemp["200d"] = np.round(dftemp["adjclose"].rolling(window = 200).mean(), 2)
    #Calculating the 50 day moving average before merging all data
    dftemp["50d"] = np.round(dftemp["adjclose"].rolling(window = 50).mean(), 2)
    #Adding a new colmn stock name to identify the stock in question
    dftemp["StockName"]=os.path.basename(file_).split('.')[0]
    alldf.append(dftemp)

df = pd.concat(alldf, axis = 0, ignore_index = True)
df.columns = ["Date", "Volume","Open", "Close", "High", "Low", "AdjustedClose","200d","50d","StockName"]
df.tail()

In [ ]:
allgoldencrossings = []
alldeathcrossings = []
index= ['Week1','Week2', 'Week3', 'Month', '2Month', '3Month', '4Month', '5Month', '6Month']
columns =['1%', '2%', '3%', '4%', '5%', '10%', '15%', '20%','30%']
for key, grp in df.groupby('StockName'): # handling data by each stock.
    previous_50 = grp['50d'].shift(1)             # shifting data by 1.
    previous_200 = grp['200d'].shift(1)           # shifting data by 1.
    deathcrossing = ((grp['50d'] <= grp['200d']) & (previous_50 > previous_200))
    goldencrossing = ((grp['50d'] >= grp['200d']) & (previous_50 < previous_200))
    allgoldencrossings.append(goldencrossing)   # Comparing current value of 50d and 200d and comparing previous value of 50d and 200d to see if the golden cross is achieved.
  # figuring out whether I am having golden cross or not.
    alldeathcrossings.append(deathcrossing)
    df['GoldenCross'] = pd.concat(allgoldencrossings)
    df['DeathCross'] = pd.concat(alldeathcrossings)
    df['WeekChange'] = df['AdjustedClose'].pct_change(periods=7)*100
    df['TwoWeekChange'] = df['AdjustedClose'].pct_change(periods=14)*100
    df['ThreeWeekChange'] = df['AdjustedClose'].pct_change(periods=21)*100
    df['MonthChange'] = df['AdjustedClose'].pct_change(periods=30)*100
    df['TwoMonthChange'] = df['AdjustedClose'].pct_change(periods=60)*100
    df['ThreeMonthChange'] = df['AdjustedClose'].pct_change(periods=90)*100
    df['FourMonthChange'] = df['AdjustedClose'].pct_change(periods=120)*100
    df['FiveMonthChange'] = df['AdjustedClose'].pct_change(periods=150)*100
    df['SixMonthChange'] = df['AdjustedClose'].pct_change(periods=180)*100

    # Count of golden crosses
goldenCount = df[df.GoldenCross].GoldenCross.count()
print("number of GoldenCrosses",goldenCount)
# Count of death crosses
deathCount = df[df.DeathCross].DeathCross.count()
print("number of DeathCrosses",deathCount)







In [ ]:
for key, grp in df.groupby('StockName'):
# Percentage of change after Golden Cross week1

    w1postiveOnePercentChange = (df[df.GoldenCross & (df.WeekChange > 1)].GoldenCross.count()/goldenCount)*100
    w1postiveTwoPercentChange = (df[df.GoldenCross & (df.WeekChange > 2)].GoldenCross.count()/goldenCount)*100
    w1postiveThreePercentChange = (df[df.GoldenCross & (df.WeekChange > 3)].GoldenCross.count()/goldenCount)*100
    w1postiveFourPercentChange = (df[df.GoldenCross & (df.WeekChange > 4)].GoldenCross.count()/goldenCount)*100
    w1postiveFivePercentChange = (df[df.GoldenCross & (df.WeekChange > 5)].GoldenCross.count()/goldenCount)*100
    w1postiveTenPercentChange = (df[df.GoldenCross & (df.WeekChange > 10)].GoldenCross.count()/goldenCount)*100
    w1postiveFifteenPercentChange = (df[df.GoldenCross & (df.WeekChange > 15)].GoldenCross.count()/goldenCount)*100
    w1postiveTwentyPercentChange = (df[df.GoldenCross & (df.WeekChange > 20)].GoldenCross.count()/goldenCount)*100
    w1postiveThirtyPercentChange = (df[df.GoldenCross & (df.WeekChange > 30)].GoldenCross.count()/goldenCount)*100
    # Percentage of change after Death Cross week1
    w1negativeOnePercentChange = (df[df.DeathCross & (df.WeekChange < 1)].DeathCross.count()/deathCount)*100
    w1negativeTwoPercentChange = (df[df.DeathCross & (df.WeekChange < 2)].DeathCross.count()/deathCount)*100
    w1negativeThreePercentChange = (df[df.DeathCross & (df.WeekChange < 3)].DeathCross.count()/deathCount)*100
    w1negativeFourPercentChange = (df[df.DeathCross & (df.WeekChange < 4)].DeathCross.count()/deathCount)*100
    w1negativeFivePercentChange = (df[df.DeathCross & (df.WeekChange < 5)].DeathCross.count()/deathCount)*100
    w1negativeTenPercentChange = (df[df.DeathCross & (df.WeekChange < 10)].DeathCross.count()/deathCount)*100
    w1negativeFifteenPercentChange = (df[df.DeathCross & (df.WeekChange < 15)].DeathCross.count()/deathCount)*100
    w1negativeTwentyPercentChange = (df[df.DeathCross & (df.WeekChange < 20)].DeathCross.count()/deathCount)*100
    w1negativeThrityPercentChange = (df[df.DeathCross & (df.WeekChange < 30)].DeathCross.count()/deathCount)*100

    #     print("1%",postiveOnePercentChange)
#     print("-1%",negativeOnePercentChange)

    # Percentage of change after Golden Cross week2
    w2postiveOnePercentChange = (df[df.GoldenCross & (df.TwoWeekChange > 1)].GoldenCross.count()/goldenCount)*100
    w2postiveTwoPercentChange = (df[df.GoldenCross & (df.TwoWeekChange > 2)].GoldenCross.count()/goldenCount)*100
    w2postiveThreePercentChange = (df[df.GoldenCross & (df.TwoWeekChange > 3)].GoldenCross.count()/goldenCount)*100
    w2postiveFourPercentChange = (df[df.GoldenCross & (df.TwoWeekChange > 4)].GoldenCross.count()/goldenCount)*100
    w2postiveFivePercentChange = (df[df.GoldenCross & (df.TwoWeekChange > 5)].GoldenCross.count()/goldenCount)*100
    w2postiveTenPercentChange = (df[df.GoldenCross & (df.TwoWeekChange > 10)].GoldenCross.count()/goldenCount)*100
    w2postiveFifteenPercentChange = (df[df.GoldenCross & (df.TwoWeekChange > 15)].GoldenCross.count()/goldenCount)*100
    w2postiveTwentyPercentChange = (df[df.GoldenCross & (df.TwoWeekChange > 20)].GoldenCross.count()/goldenCount)*100
    w2postiveThirtyPercentChange = (df[df.GoldenCross & (df.TwoWeekChange > 30)].GoldenCross.count()/goldenCount)*100

    # Percentage of change after Death Cross week2
    w2negativeOnePercentChange = (df[df.DeathCross & (df.TwoWeekChange < 1)].DeathCross.count()/deathCount)*100
    w2negativeTwoPercentChange = (df[df.DeathCross & (df.TwoWeekChange < 2)].DeathCross.count()/deathCount)*100
    w2negativeThreePercentChange = (df[df.DeathCross & (df.TwoWeekChange < 3)].DeathCross.count()/deathCount)*100
    w2negativeFourPercentChange = (df[df.DeathCross & (df.TwoWeekChange < 4)].DeathCross.count()/deathCount)*100
    w2negativeFivePercentChange = (df[df.DeathCross & (df.TwoWeekChange < 5)].DeathCross.count()/deathCount)*100
    w2negativeTenPercentChange = (df[df.DeathCross & (df.TwoWeekChange < 10)].DeathCross.count()/deathCount)*100
    w2negativeFifteenPercentChange = (df[df.DeathCross & (df.TwoWeekChange < 15)].DeathCross.count()/deathCount)*100
    w2negativeTwentyPercentChange = (df[df.DeathCross & (df.TwoWeekChange < 20)].DeathCross.count()/deathCount)*100
    w2negativeThirtyPercentChange = (df[df.DeathCross & (df.TwoWeekChange < 30)].DeathCross.count()/deathCount)*100

#     print(postiveTwoPercentChange)
#     print(negativeTwoPercentChange)

    # Percentage of change after Golden Cross week3
    w3postiveOnePercentChange = (df[df.GoldenCross & (df.ThreeWeekChange > 1)].GoldenCross.count()/goldenCount)*100
    w3postiveTwoPercentChange = (df[df.GoldenCross & (df.ThreeWeekChange > 2)].GoldenCross.count()/goldenCount)*100
    w3postiveThreePercentChange = (df[df.GoldenCross & (df.ThreeWeekChange > 3)].GoldenCross.count()/goldenCount)*100
    w3postiveFourPercentChange = (df[df.GoldenCross & (df.ThreeWeekChange > 4)].GoldenCross.count()/goldenCount)*100
    w3postiveFivePercentChange = (df[df.GoldenCross & (df.ThreeWeekChange > 5)].GoldenCross.count()/goldenCount)*100
    w3postiveTenPercentChange = (df[df.GoldenCross & (df.ThreeWeekChange > 10)].GoldenCross.count()/goldenCount)*100
    w3postiveFifteenPercentChange = (df[df.GoldenCross & (df.ThreeWeekChange > 15)].GoldenCross.count()/goldenCount)*100
    w3postiveTwentyPercentChange = (df[df.GoldenCross & (df.ThreeWeekChange > 20)].GoldenCross.count()/goldenCount)*100
    w3postiveThirtyPercentChange = (df[df.GoldenCross & (df.ThreeWeekChange > 30)].GoldenCross.count()/goldenCount)*100


    # Percentage of change after Death Cross week3
    w3negativeOnePercentChange = (df[df.DeathCross & (df.ThreeWeekChange < 1)].DeathCross.count()/deathCount)*100
    w3negativeTwoPercentChange = (df[df.DeathCross & (df.ThreeWeekChange < 2)].DeathCross.count()/deathCount)*100
    w3negativeThreePercentChange = (df[df.DeathCross & (df.ThreeWeekChange < 3)].DeathCross.count()/deathCount)*100
    w3negativeFourPercentChange = (df[df.DeathCross & (df.ThreeWeekChange < 4)].DeathCross.count()/deathCount)*100
    w3negativeFivePercentChange = (df[df.DeathCross & (df.ThreeWeekChange < 5)].DeathCross.count()/deathCount)*100
    w3negativeTenPercentChange = (df[df.DeathCross & (df.ThreeWeekChange < 10)].DeathCross.count()/deathCount)*100
    w3negativeFifteenPercentChange = (df[df.DeathCross & (df.ThreeWeekChange < 15)].DeathCross.count()/deathCount)*100
    w3negativeTwentyPercentChange = (df[df.DeathCross & (df.ThreeWeekChange < 20)].DeathCross.count()/deathCount)*100
    w3negativeThirtyPercentChange = (df[df.DeathCross & (df.ThreeWeekChange < 30)].DeathCross.count()/deathCount)*100

#     print(postiveThreePercentChange)
#     print(negativeThreePercentChange)

    # Percentage of change in Golden Cross in a month
    m1postiveOnePercentChange = (df[df.GoldenCross & (df.MonthChange > 1)].GoldenCross.count()/goldenCount)*100
    m1postiveTwoPercentChange = (df[df.GoldenCross & (df.MonthChange > 2)].GoldenCross.count()/goldenCount)*100
    m1postiveThreePercentChange = (df[df.GoldenCross & (df.MonthChange > 3)].GoldenCross.count()/goldenCount)*100
    m1postiveFourPercentChange = (df[df.GoldenCross & (df.MonthChange > 4)].GoldenCross.count()/goldenCount)*100
    m1postiveFivePercentChange = (df[df.GoldenCross & (df.MonthChange > 5)].GoldenCross.count()/goldenCount)*100
    m1postiveTenPercentChange = (df[df.GoldenCross & (df.MonthChange > 10)].GoldenCross.count()/goldenCount)*100
    m1postiveFifteenPercentChange = (df[df.GoldenCross & (df.MonthChange > 15)].GoldenCross.count()/goldenCount)*100
    m1postiveTwentyPercentChange = (df[df.GoldenCross & (df.MonthChange > 20)].GoldenCross.count()/goldenCount)*100
    m1postiveThirtyPercentChange = (df[df.GoldenCross & (df.MonthChange > 30)].GoldenCross.count()/goldenCount)*100

    # Percentage of change after Death Cross in a month
    m1negativeOnePercentChange = (df[df.DeathCross & (df.MonthChange < 1)].DeathCross.count()/deathCount)*100
    m1negativeTwoPercentChange = (df[df.DeathCross & (df.MonthChange < 2)].DeathCross.count()/deathCount)*100
    m1negativeThreePercentChange = (df[df.DeathCross & (df.MonthChange < 3)].DeathCross.count()/deathCount)*100
    m1negativeFourPercentChange = (df[df.DeathCross & (df.MonthChange < 4)].DeathCross.count()/deathCount)*100
    m1negativeFivePercentChange = (df[df.DeathCross & (df.MonthChange < 5)].DeathCross.count()/deathCount)*100
    m1negativeTenPercentChange = (df[df.DeathCross & (df.MonthChange < 10)].DeathCross.count()/deathCount)*100
    m1negativeFifteenPercentChange = (df[df.DeathCross & (df.MonthChange < 15)].DeathCross.count()/deathCount)*100
    m1negativeTwentyPercentChange = (df[df.DeathCross & (df.MonthChange < 20)].DeathCross.count()/deathCount)*100
    m1negativeThirtyPercentChange = (df[df.DeathCross & (df.MonthChange < 30)].DeathCross.count()/deathCount)*100

#     print(postiveFourPercentChange)
#     print(negativeFourPercentChange)

    # Percentage of change after Golden Cross in two months
    m2postiveOnePercentChange = (df[df.GoldenCross & (df.TwoMonthChange > 1)].GoldenCross.count()/goldenCount)*100
    m2postiveTwoPercentChange = (df[df.GoldenCross & (df.TwoMonthChange > 2)].GoldenCross.count()/goldenCount)*100
    m2postiveThreePercentChange = (df[df.GoldenCross & (df.TwoMonthChange > 3)].GoldenCross.count()/goldenCount)*100
    m2postiveFourPercentChange = (df[df.GoldenCross & (df.TwoMonthChange > 4)].GoldenCross.count()/goldenCount)*100
    m2postiveFivePercentChange = (df[df.GoldenCross & (df.TwoMonthChange > 5)].GoldenCross.count()/goldenCount)*100
    m2postiveTenPercentChange = (df[df.GoldenCross & (df.TwoMonthChange > 10)].GoldenCross.count()/goldenCount)*100
    m2postiveFifteenPercentChange = (df[df.GoldenCross & (df.TwoMonthChange > 15)].GoldenCross.count()/goldenCount)*100
    m2postiveTwentyPercentChange = (df[df.GoldenCross & (df.TwoMonthChange > 20)].GoldenCross.count()/goldenCount)*100
    m2postiveThirtyPercentChange = (df[df.GoldenCross & (df.TwoMonthChange > 30)].GoldenCross.count()/goldenCount)*100

    # Percentage of change after Death Cross in two months
    m2negativeOnePercentChange = (df[df.DeathCross & (df.TwoMonthChange < 1)].DeathCross.count()/deathCount)*100
    m2negativeTwoPercentChange = (df[df.DeathCross & (df.TwoMonthChange < 2)].DeathCross.count()/deathCount)*100
    m2negativeThreePercentChange = (df[df.DeathCross & (df.TwoMonthChange < 3)].DeathCross.count()/deathCount)*100
    m2negativeFourPercentChange = (df[df.DeathCross & (df.TwoMonthChange < 4)].DeathCross.count()/deathCount)*100
    m2negativeFivePercentChange = (df[df.DeathCross & (df.TwoMonthChange < 5)].DeathCross.count()/deathCount)*100
    m2negativeTenPercentChange = (df[df.DeathCross & (df.TwoMonthChange < 10)].DeathCross.count()/deathCount)*100
    m2negativeFifteenPercentChange = (df[df.DeathCross & (df.TwoMonthChange < 15)].DeathCross.count()/deathCount)*100
    m2negativeTwentyPercentChange = (df[df.DeathCross & (df.TwoMonthChange < 20)].DeathCross.count()/deathCount)*100
    m2negativeThirtyPercentChange = (df[df.DeathCross & (df.TwoMonthChange < 30)].DeathCross.count()/deathCount)*100

#     print(postiveFivePercentChange)
#     print(negativeFivePercentChange)

    # Percentage of change after Golden Cross in three months
    m3postiveOnePercentChange = (df[df.GoldenCross & (df.ThreeMonthChange > 1)].GoldenCross.count()/goldenCount)*100
    m3postiveTwoPercentChange = (df[df.GoldenCross & (df.ThreeMonthChange > 2)].GoldenCross.count()/goldenCount)*100
    m3postiveThreePercentChange = (df[df.GoldenCross & (df.ThreeMonthChange > 3)].GoldenCross.count()/goldenCount)*100
    m3postiveFourPercentChange = (df[df.GoldenCross & (df.ThreeMonthChange > 4)].GoldenCross.count()/goldenCount)*100
    m3postiveFivePercentChange = (df[df.GoldenCross & (df.ThreeMonthChange > 5)].GoldenCross.count()/goldenCount)*100
    m3postiveTenPercentChange = (df[df.GoldenCross & (df.ThreeMonthChange > 10)].GoldenCross.count()/goldenCount)*100
    m3postiveFifteenPercentChange = (df[df.GoldenCross & (df.ThreeMonthChange > 15)].GoldenCross.count()/goldenCount)*100
    m3postiveTwentyPercentChange = (df[df.GoldenCross & (df.ThreeMonthChange > 20)].GoldenCross.count()/goldenCount)*100
    m3postiveThirtyPercentChange = (df[df.GoldenCross & (df.ThreeMonthChange > 30)].GoldenCross.count()/goldenCount)*100

    
    # Percentage of change after Death Cross in three months
    m3negativeOnePercentChange = (df[df.DeathCross & (df.ThreeMonthChange < 1)].DeathCross.count()/deathCount)*100
    m3negativeTwoPercentChange = (df[df.DeathCross & (df.ThreeMonthChange < 2)].DeathCross.count()/deathCount)*100
    m3negativeThreePercentChange = (df[df.DeathCross & (df.ThreeMonthChange < 3)].DeathCross.count()/deathCount)*100
    m3negativeFourPercentChange = (df[df.DeathCross & (df.ThreeMonthChange < 4)].DeathCross.count()/deathCount)*100
    m3negativeFivePercentChange = (df[df.DeathCross & (df.ThreeMonthChange < 5)].DeathCross.count()/deathCount)*100
    m3negativeTenPercentChange = (df[df.DeathCross & (df.ThreeMonthChange < 10)].DeathCross.count()/deathCount)*100
    m3negativeFifteenPercentChange = (df[df.DeathCross & (df.ThreeMonthChange < 15)].DeathCross.count()/deathCount)*100
    m3negativeTwentyPercentChange = (df[df.DeathCross & (df.ThreeMonthChange < 20)].DeathCross.count()/deathCount)*100
    m3negativeThirtyPercentChange = (df[df.DeathCross & (df.ThreeMonthChange < 30)].DeathCross.count()/deathCount)*100


    
#     print(postiveTenPercentChange)
#     print(negativeTenPercentChange)

    # Percentage of change after Golden Cross in Four months
    m4postiveOnePercentChange = (df[df.GoldenCross & (df.FourMonthChange > 1)].GoldenCross.count()/goldenCount)*100
    m4postiveTwoPercentChange = (df[df.GoldenCross & (df.FourMonthChange > 2)].GoldenCross.count()/goldenCount)*100
    m4postiveThreePercentChange = (df[df.GoldenCross & (df.FourMonthChange > 3)].GoldenCross.count()/goldenCount)*100
    m4postiveFourPercentChange = (df[df.GoldenCross & (df.FourMonthChange > 4)].GoldenCross.count()/goldenCount)*100
    m4postiveFivePercentChange = (df[df.GoldenCross & (df.FourMonthChange > 5)].GoldenCross.count()/goldenCount)*100
    m4postiveTenPercentChange = (df[df.GoldenCross & (df.FourMonthChange > 10)].GoldenCross.count()/goldenCount)*100
    m4postiveFifteenPercentChange = (df[df.GoldenCross & (df.FourMonthChange > 15)].GoldenCross.count()/goldenCount)*100
    m4postiveTwentyPercentChange = (df[df.GoldenCross & (df.FourMonthChange > 20)].GoldenCross.count()/goldenCount)*100
    m4postiveThirtyPercentChange = (df[df.GoldenCross & (df.FourMonthChange > 30)].GoldenCross.count()/goldenCount)*100


    # Percentage of change after Death Cross in four months
    m4negativeOnePercentChange = (df[df.DeathCross & (df.FourMonthChange < 1)].DeathCross.count()/deathCount)*100
    m4negativeTwoPercentChange = (df[df.DeathCross & (df.FourMonthChange < 2)].DeathCross.count()/deathCount)*100
    m4negativeThreePercentChange = (df[df.DeathCross & (df.FourMonthChange < 3)].DeathCross.count()/deathCount)*100
    m4negativeFourPercentChange = (df[df.DeathCross & (df.FourMonthChange < 4)].DeathCross.count()/deathCount)*100
    m4negativeFivePercentChange = (df[df.DeathCross & (df.FourMonthChange < 5)].DeathCross.count()/deathCount)*100
    m4negativeTenPercentChange = (df[df.DeathCross & (df.FourMonthChange < 10)].DeathCross.count()/deathCount)*100
    m4negativeFifteenPercentChange = (df[df.DeathCross & (df.FourMonthChange < 15)].DeathCross.count()/deathCount)*100
    m4negativeTwentyPercentChange = (df[df.DeathCross & (df.FourMonthChange < 20)].DeathCross.count()/deathCount)*100
    m4negativeThirtyPercentChange = (df[df.DeathCross & (df.FourMonthChange < 30)].DeathCross.count()/deathCount)*100

    
    
    #     print(postiveFifPercentChange)
#     print(negativeFifPercentChange)

    # Percentage of change after Golden Cross in five months
    m5postiveOnePercentChange = (df[df.GoldenCross & (df.FiveMonthChange > 1)].GoldenCross.count()/goldenCount)*100
    m5postiveTwoPercentChange = (df[df.GoldenCross & (df.FiveMonthChange > 2)].GoldenCross.count()/goldenCount)*100
    m5postiveThreePercentChange = (df[df.GoldenCross & (df.FiveMonthChange > 3)].GoldenCross.count()/goldenCount)*100
    m5postiveFourPercentChange = (df[df.GoldenCross & (df.FiveMonthChange > 4)].GoldenCross.count()/goldenCount)*100
    m5postiveFivePercentChange = (df[df.GoldenCross & (df.FiveMonthChange > 5)].GoldenCross.count()/goldenCount)*100
    m5postiveTenPercentChange = (df[df.GoldenCross & (df.FiveMonthChange > 10)].GoldenCross.count()/goldenCount)*100
    m5postiveFifteenPercentChange = (df[df.GoldenCross & (df.FiveMonthChange > 15)].GoldenCross.count()/goldenCount)*100
    m5postiveTwentyPercentChange = (df[df.GoldenCross & (df.FiveMonthChange > 20)].GoldenCross.count()/goldenCount)*100
    m5postiveThirtyPercentChange = (df[df.GoldenCross & (df.FiveMonthChange > 30)].GoldenCross.count()/goldenCount)*100

    # Percentage of change after Death Cross in Fivemonths months
    m5negativeOnePercentChange = (df[df.DeathCross & (df.FiveMonthChange < 1)].DeathCross.count()/deathCount)*100
    m5negativeTwoPercentChange = (df[df.DeathCross & (df.FiveMonthChange < 2)].DeathCross.count()/deathCount)*100
    m5negativeThreePercentChange = (df[df.DeathCross & (df.FiveMonthChange < 3)].DeathCross.count()/deathCount)*100
    m5negativeFourPercentChange = (df[df.DeathCross & (df.FiveMonthChange < 4)].DeathCross.count()/deathCount)*100
    m5negativeFivePercentChange = (df[df.DeathCross & (df.FiveMonthChange < 5)].DeathCross.count()/deathCount)*100
    m5negativeTenPercentChange = (df[df.DeathCross & (df.FiveMonthChange < 10)].DeathCross.count()/deathCount)*100
    m5negativeFifteenPercentChange = (df[df.DeathCross & (df.FiveMonthChange < 15)].DeathCross.count()/deathCount)*100
    m5negativeTwentyPercentChange = (df[df.DeathCross & (df.FiveMonthChange < 20)].DeathCross.count()/deathCount)*100
    m5negativeThirtyPercentChange = (df[df.DeathCross & (df.FiveMonthChange < 30)].DeathCross.count()/deathCount)*100

    
    
    #     print(postiveTwenPercentChange)
#     print(negativeTwenPercentChange)
    m6postiveOnePercentChange = (df[df.GoldenCross & (df.SixMonthChange > 1)].GoldenCross.count()/goldenCount)*100
    m6postiveTwoPercentChange = (df[df.GoldenCross & (df.SixMonthChange > 2)].GoldenCross.count()/goldenCount)*100
    m6postiveThreePercentChange = (df[df.GoldenCross & (df.SixMonthChange > 3)].GoldenCross.count()/goldenCount)*100
    m6postiveFourPercentChange = (df[df.GoldenCross & (df.SixMonthChange > 4)].GoldenCross.count()/goldenCount)*100
    m6postiveFivePercentChange = (df[df.GoldenCross & (df.SixMonthChange > 5)].GoldenCross.count()/goldenCount)*100
    m6postiveTenPercentChange = (df[df.GoldenCross & (df.SixMonthChange > 10)].GoldenCross.count()/goldenCount)*100
    m6postiveFifteenPercentChange = (df[df.GoldenCross & (df.SixMonthChange > 15)].GoldenCross.count()/goldenCount)*100
    m6postiveTwentyPercentChange = (df[df.GoldenCross & (df.SixMonthChange > 20)].GoldenCross.count()/goldenCount)*100
    m6postiveThirtyPercentChange = (df[df.GoldenCross & (df.SixMonthChange > 30)].GoldenCross.count()/goldenCount)*100

    m6negativeOnePercentChange = (df[df.DeathCross & (df.SixMonthChange < 1)].DeathCross.count()/deathCount)*100
    m6negativeTwoPercentChange = (df[df.DeathCross & (df.SixMonthChange < 2)].DeathCross.count()/deathCount)*100
    m6negativeThreePercentChange = (df[df.DeathCross & (df.SixMonthChange < 3)].DeathCross.count()/deathCount)*100
    m6negativeFourPercentChange = (df[df.DeathCross & (df.SixMonthChange < 4)].DeathCross.count()/deathCount)*100
    m6negativeFivePercentChange = (df[df.DeathCross & (df.SixMonthChange < 5)].DeathCross.count()/deathCount)*100
    m6negativeTenPercentChange = (df[df.DeathCross & (df.SixMonthChange < 10)].DeathCross.count()/deathCount)*100
    m6negativeFifteenPercentChange = (df[df.DeathCross & (df.SixMonthChange < 15)].DeathCross.count()/deathCount)*100
    m6negativeTwentyPercentChange = (df[df.DeathCross & (df.SixMonthChange < 20)].DeathCross.count()/deathCount)*100
    m6negativeThirtyPercentChange = (df[df.DeathCross & (df.SixMonthChange < 30)].DeathCross.count()/deathCount)*100


    Analysis = pd.DataFrame(index=index, columns=columns)
    DeathAnalysis = pd.DataFrame(index=index, columns=columns)
    #Week1 Row all percentages after Golden cross
    Analysis.loc[Analysis.index[0], '1%'] = w1postiveOnePercentChange
    Analysis.loc[Analysis.index[0], '2%'] = w1postiveTwoPercentChange
    Analysis.loc[Analysis.index[0], '3%'] = w1postiveThreePercentChange
    Analysis.loc[Analysis.index[0], '4%'] = w1postiveFourPercentChange
    Analysis.loc[Analysis.index[0], '5%'] = w1postiveFivePercentChange
    Analysis.loc[Analysis.index[0], '10%'] = w1postiveTenPercentChange
    Analysis.loc[Analysis.index[0], '15%'] = w1postiveFifteenPercentChange
    Analysis.loc[Analysis.index[0], '20%'] = w1postiveTwentyPercentChange
    Analysis.loc[Analysis.index[0], '30%'] = w1postiveThirtyPercentChange

    #Week2 Row all percentages after Golden cross
    Analysis.loc[Analysis.index[1], '1%'] = w2postiveOnePercentChange
    Analysis.loc[Analysis.index[1], '2%'] = w2postiveTwoPercentChange
    Analysis.loc[Analysis.index[1], '3%'] = w2postiveThreePercentChange
    Analysis.loc[Analysis.index[1], '4%'] = w2postiveFourPercentChange
    Analysis.loc[Analysis.index[1], '5%'] = w2postiveFivePercentChange
    Analysis.loc[Analysis.index[1], '10%'] = w2postiveTenPercentChange
    Analysis.loc[Analysis.index[1], '15%'] = w2postiveFifteenPercentChange
    Analysis.loc[Analysis.index[1], '20%'] = w2postiveTwentyPercentChange
    Analysis.loc[Analysis.index[1], '30%'] = w2postiveThirtyPercentChange

    #Week3 Row all percentages after Golden cross
    Analysis.loc[Analysis.index[2], '1%'] = w3postiveOnePercentChange
    Analysis.loc[Analysis.index[2], '2%'] = w3postiveTwoPercentChange
    Analysis.loc[Analysis.index[2], '3%'] = w3postiveThreePercentChange
    Analysis.loc[Analysis.index[2], '4%'] = w3postiveFourPercentChange
    Analysis.loc[Analysis.index[2], '5%'] = w3postiveFivePercentChange
    Analysis.loc[Analysis.index[2], '10%'] = w3postiveTenPercentChange
    Analysis.loc[Analysis.index[2], '15%'] = w3postiveFifteenPercentChange
    Analysis.loc[Analysis.index[2], '20%'] = w3postiveTwentyPercentChange
    Analysis.loc[Analysis.index[2], '30%'] = w3postiveThirtyPercentChange

    #Month Row all percentages after Golden cross
    Analysis.loc[Analysis.index[3], '1%'] = m1postiveOnePercentChange
    Analysis.loc[Analysis.index[3], '2%'] = m1postiveTwoPercentChange
    Analysis.loc[Analysis.index[3], '3%'] = m1postiveThreePercentChange
    Analysis.loc[Analysis.index[3], '4%'] = m1postiveFourPercentChange
    Analysis.loc[Analysis.index[3], '5%'] = m1postiveFivePercentChange
    Analysis.loc[Analysis.index[3], '10%'] = m1postiveTenPercentChange
    Analysis.loc[Analysis.index[3], '15%'] = m1postiveFifteenPercentChange
    Analysis.loc[Analysis.index[3], '20%'] = m1postiveTwentyPercentChange
    Analysis.loc[Analysis.index[3], '30%'] = m1postiveThirtyPercentChange

    #Twomonths Row all percentages after Golden cross
    Analysis.loc[Analysis.index[4], '1%'] = m2postiveOnePercentChange
    Analysis.loc[Analysis.index[4], '2%'] = m2postiveTwoPercentChange
    Analysis.loc[Analysis.index[4], '3%'] = m2postiveThreePercentChange
    Analysis.loc[Analysis.index[4], '4%'] = m2postiveFourPercentChange
    Analysis.loc[Analysis.index[4], '5%'] = m2postiveFivePercentChange
    Analysis.loc[Analysis.index[4], '10%'] = m2postiveTenPercentChange
    Analysis.loc[Analysis.index[4], '15%'] = m2postiveFifteenPercentChange
    Analysis.loc[Analysis.index[4], '20%'] = m2postiveTwentyPercentChange
    Analysis.loc[Analysis.index[4], '30%'] = m2postiveThirtyPercentChange
    #Threemonths Row all percentages after Golden cross
    Analysis.loc[Analysis.index[5], '1%'] = m3postiveOnePercentChange
    Analysis.loc[Analysis.index[5], '2%'] = m3postiveTwoPercentChange
    Analysis.loc[Analysis.index[5], '3%'] = m3postiveThreePercentChange
    Analysis.loc[Analysis.index[5], '4%'] = m3postiveFourPercentChange
    Analysis.loc[Analysis.index[5], '5%'] = m3postiveFivePercentChange
    Analysis.loc[Analysis.index[5], '10%'] = m3postiveTenPercentChange
    Analysis.loc[Analysis.index[5], '15%'] = m3postiveFifteenPercentChange
    Analysis.loc[Analysis.index[5], '20%'] = m3postiveTwentyPercentChange
    Analysis.loc[Analysis.index[5], '30%'] = m3postiveThirtyPercentChange

    #Fourmonths Row all percentages after Golden cross
    Analysis.loc[Analysis.index[6], '1%'] = m4postiveOnePercentChange
    Analysis.loc[Analysis.index[6], '2%'] = m4postiveTwoPercentChange
    Analysis.loc[Analysis.index[6], '3%'] = m4postiveThreePercentChange
    Analysis.loc[Analysis.index[6], '4%'] = m4postiveFourPercentChange
    Analysis.loc[Analysis.index[6], '5%'] = m4postiveFivePercentChange
    Analysis.loc[Analysis.index[6], '10%'] = m4postiveTenPercentChange
    Analysis.loc[Analysis.index[6], '15%'] = m4postiveFifteenPercentChange
    Analysis.loc[Analysis.index[6], '20%'] = m4postiveTwentyPercentChange
    Analysis.loc[Analysis.index[6], '30%'] = m4postiveThirtyPercentChange


    #Fivemonths Row all percentages after Golden cross
    Analysis.loc[Analysis.index[7], '1%'] = m5postiveOnePercentChange
    Analysis.loc[Analysis.index[7], '2%'] = m5postiveTwoPercentChange
    Analysis.loc[Analysis.index[7], '3%'] = m5postiveThreePercentChange
    Analysis.loc[Analysis.index[7], '4%'] = m5postiveFourPercentChange
    Analysis.loc[Analysis.index[7], '5%'] = m5postiveFivePercentChange
    Analysis.loc[Analysis.index[7], '10%'] = m5postiveTenPercentChange
    Analysis.loc[Analysis.index[7], '15%'] = m5postiveFifteenPercentChange
    Analysis.loc[Analysis.index[7], '20%'] = m5postiveTwentyPercentChange
    Analysis.loc[Analysis.index[7], '30%'] = m5postiveThirtyPercentChange


    #sixmonths Row all percentages after Golden cross
    Analysis.loc[Analysis.index[8], '1%'] = m6postiveOnePercentChange
    Analysis.loc[Analysis.index[8], '2%'] = m6postiveTwoPercentChange
    Analysis.loc[Analysis.index[8], '3%'] = m6postiveThreePercentChange
    Analysis.loc[Analysis.index[8], '4%'] = m6postiveFourPercentChange
    Analysis.loc[Analysis.index[8], '5%'] = m6postiveFivePercentChange
    Analysis.loc[Analysis.index[8], '10%'] = m6postiveTenPercentChange
    Analysis.loc[Analysis.index[8], '15%'] = m6postiveFifteenPercentChange
    Analysis.loc[Analysis.index[8], '20%'] = m6postiveTwentyPercentChange
    Analysis.loc[Analysis.index[8], '30%'] = m6postiveThirtyPercentChange


    print(Analysis)

    #Week1 Row all percentages after Death cross
    DeathAnalysis.loc[DeathAnalysis.index[0], '1%'] = w1negativeOnePercentChange
    DeathAnalysis.loc[DeathAnalysis.index[0], '2%'] = w1negativeTwoPercentChange
    DeathAnalysis.loc[DeathAnalysis.index[0], '3%'] = w1negativeThreePercentChange
    DeathAnalysis.loc[DeathAnalysis.index[0], '4%'] = w1negativeFourPercentChange
    DeathAnalysis.loc[DeathAnalysis.index[0], '5%'] = w1negativeFivePercentChange
    DeathAnalysis.loc[DeathAnalysis.index[0], '10%'] = w1negativeTenPercentChange
    DeathAnalysis.loc[DeathAnalysis.index[0], '15%'] = w1negativeFifteenPercentChange
    DeathAnalysis.loc[DeathAnalysis.index[0], '20%'] = w1negativeTwentyPercentChange
    DeathAnalysis.loc[DeathAnalysis.index[0], '30%'] = w1negativeThrityPercentChange
    #Week2 Row all percentages after Death cross
    DeathAnalysis.loc[DeathAnalysis.index[1], '1%'] = w2negativeOnePercentChange
    DeathAnalysis.loc[DeathAnalysis.index[1], '2%'] = w2negativeTwoPercentChange
    DeathAnalysis.loc[DeathAnalysis.index[1], '3%'] = w2negativeThreePercentChange
    DeathAnalysis.loc[DeathAnalysis.index[1], '4%'] = w2negativeFourPercentChange
    DeathAnalysis.loc[DeathAnalysis.index[1], '5%'] = w2negativeFivePercentChange
    DeathAnalysis.loc[DeathAnalysis.index[1], '10%'] = w2negativeTenPercentChange
    DeathAnalysis.loc[DeathAnalysis.index[1], '15%'] = w2negativeFifteenPercentChange
    DeathAnalysis.loc[DeathAnalysis.index[1], '20%'] = w2negativeTwentyPercentChange
    DeathAnalysis.loc[DeathAnalysis.index[1], '30%'] = w2negativeThirtyPercentChange
    #Week3 Row all percentages after Death cross
    DeathAnalysis.loc[DeathAnalysis.index[2], '1%'] = w3negativeOnePercentChange
    DeathAnalysis.loc[DeathAnalysis.index[2], '2%'] = w3negativeTwoPercentChange
    DeathAnalysis.loc[DeathAnalysis.index[2], '3%'] = w3negativeThreePercentChange
    DeathAnalysis.loc[DeathAnalysis.index[2], '4%'] = w3negativeFourPercentChange
    DeathAnalysis.loc[DeathAnalysis.index[2], '5%'] = w3negativeFivePercentChange
    DeathAnalysis.loc[DeathAnalysis.index[2], '10%'] = w3negativeTenPercentChange
    DeathAnalysis.loc[DeathAnalysis.index[2], '15%'] = w3negativeFifteenPercentChange
    DeathAnalysis.loc[DeathAnalysis.index[2], '20%'] = w3negativeTwentyPercentChange
    DeathAnalysis.loc[DeathAnalysis.index[2], '30%'] = w3negativeThirtyPercentChange

    #month Row all percentages after Death cross
    DeathAnalysis.loc[DeathAnalysis.index[3], '1%'] = m1negativeOnePercentChange
    DeathAnalysis.loc[DeathAnalysis.index[3], '2%'] = m1negativeTwoPercentChange
    DeathAnalysis.loc[DeathAnalysis.index[3], '3%'] = m1negativeThreePercentChange
    DeathAnalysis.loc[DeathAnalysis.index[3], '4%'] = m1negativeFourPercentChange
    DeathAnalysis.loc[DeathAnalysis.index[3], '5%'] = m1negativeFivePercentChange
    DeathAnalysis.loc[DeathAnalysis.index[3], '10%'] = m1negativeTenPercentChange
    DeathAnalysis.loc[DeathAnalysis.index[3], '15%'] = m1negativeFifteenPercentChange
    DeathAnalysis.loc[DeathAnalysis.index[3], '20%'] = m1negativeTwentyPercentChange
    DeathAnalysis.loc[DeathAnalysis.index[3], '30%'] = m1negativeThirtyPercentChange
    #Twomonths Row all percentages after Death cross
    DeathAnalysis.loc[DeathAnalysis.index[4], '1%'] = m2negativeOnePercentChange
    DeathAnalysis.loc[DeathAnalysis.index[4], '2%'] = m2negativeTwoPercentChange
    DeathAnalysis.loc[DeathAnalysis.index[4], '3%'] = m2negativeThreePercentChange
    DeathAnalysis.loc[DeathAnalysis.index[4], '4%'] = m2negativeFourPercentChange
    DeathAnalysis.loc[DeathAnalysis.index[4], '5%'] = m2negativeFivePercentChange
    DeathAnalysis.loc[DeathAnalysis.index[4], '10%'] = m2negativeTenPercentChange
    DeathAnalysis.loc[DeathAnalysis.index[4], '15%'] = m2negativeFifteenPercentChange
    DeathAnalysis.loc[DeathAnalysis.index[4], '20%'] = m2negativeTwentyPercentChange
    DeathAnalysis.loc[DeathAnalysis.index[4], '30%'] = m2negativeThirtyPercentChange
    #Threeomonths Row all percentages after Death cross
    DeathAnalysis.loc[DeathAnalysis.index[5], '1%'] = m3negativeOnePercentChange
    DeathAnalysis.loc[DeathAnalysis.index[5], '2%'] = m3negativeTwoPercentChange
    DeathAnalysis.loc[DeathAnalysis.index[5], '3%'] = m3negativeThreePercentChange
    DeathAnalysis.loc[DeathAnalysis.index[5], '4%'] = m3negativeFourPercentChange
    DeathAnalysis.loc[DeathAnalysis.index[5], '5%'] = m3negativeFivePercentChange
    DeathAnalysis.loc[DeathAnalysis.index[5], '10%'] = m3negativeTenPercentChange
    DeathAnalysis.loc[DeathAnalysis.index[5], '15%'] = m3negativeFifteenPercentChange
    DeathAnalysis.loc[DeathAnalysis.index[5], '20%'] = m3negativeTwentyPercentChange
    DeathAnalysis.loc[DeathAnalysis.index[5], '30%'] = m3negativeThirtyPercentChange
    #Fourmonths Row all percentages after Death cross
    DeathAnalysis.loc[DeathAnalysis.index[6], '1%'] = m4negativeOnePercentChange
    DeathAnalysis.loc[DeathAnalysis.index[6], '2%'] = m4negativeTwoPercentChange
    DeathAnalysis.loc[DeathAnalysis.index[6], '3%'] = m4negativeThreePercentChange
    DeathAnalysis.loc[DeathAnalysis.index[6], '4%'] = m4negativeFourPercentChange
    DeathAnalysis.loc[DeathAnalysis.index[6], '5%'] = m4negativeFivePercentChange
    DeathAnalysis.loc[DeathAnalysis.index[6], '10%'] = m4negativeTenPercentChange
    DeathAnalysis.loc[DeathAnalysis.index[6], '15%'] = m4negativeFifteenPercentChange
    DeathAnalysis.loc[DeathAnalysis.index[6], '20%'] = m4negativeTwentyPercentChange
    DeathAnalysis.loc[DeathAnalysis.index[6], '30%'] = m4negativeThirtyPercentChange

    #Fivemonths Row all percentages after Death cross
    DeathAnalysis.loc[DeathAnalysis.index[7], '1%'] = m5negativeOnePercentChange
    DeathAnalysis.loc[DeathAnalysis.index[7], '2%'] = m5negativeTwoPercentChange
    DeathAnalysis.loc[DeathAnalysis.index[7], '3%'] = m5negativeThreePercentChange
    DeathAnalysis.loc[DeathAnalysis.index[7], '4%'] = m5negativeFourPercentChange
    DeathAnalysis.loc[DeathAnalysis.index[7], '5%'] = m5negativeFivePercentChange
    DeathAnalysis.loc[DeathAnalysis.index[7], '10%'] = m5negativeTenPercentChange
    DeathAnalysis.loc[DeathAnalysis.index[7], '15%'] = m5negativeFifteenPercentChange
    DeathAnalysis.loc[DeathAnalysis.index[7], '20%'] = m5negativeTwentyPercentChange
    DeathAnalysis.loc[DeathAnalysis.index[7], '30%'] = m5negativeThirtyPercentChange

    #Sixmonths Row all percentages after Death cross
    DeathAnalysis.loc[DeathAnalysis.index[8], '1%'] = m6negativeOnePercentChange
    DeathAnalysis.loc[DeathAnalysis.index[8], '2%'] = m6negativeTwoPercentChange
    DeathAnalysis.loc[DeathAnalysis.index[8], '3%'] = m6negativeThreePercentChange
    DeathAnalysis.loc[DeathAnalysis.index[8], '4%'] = m6negativeFourPercentChange
    DeathAnalysis.loc[DeathAnalysis.index[8], '5%'] = m6negativeFivePercentChange
    DeathAnalysis.loc[DeathAnalysis.index[8], '10%'] = m6negativeTenPercentChange
    DeathAnalysis.loc[DeathAnalysis.index[8], '15%'] = m6negativeFifteenPercentChange
    DeathAnalysis.loc[DeathAnalysis.index[8], '20%'] = m6negativeTwentyPercentChange
    DeathAnalysis.loc[DeathAnalysis.index[8], '30%'] = m6negativeThirtyPercentChange
    print(DeathAnalysis)



In [ ]:
stocklist = ['BA','DXC','AA','CR','CUB','CAR','AFL']
for key, grp in df.groupby('StockName'):  
    if key in stocklist:
    # Percentage of change after Golden Cross week1
        print(key)
        w1postiveOnePercentChange = (df[df.GoldenCross & (df.WeekChange > 1)].GoldenCross.count()/goldenCount)*100
        w1postiveTwoPercentChange = (df[df.GoldenCross & (df.WeekChange > 2)].GoldenCross.count()/goldenCount)*100
        w1postiveThreePercentChange = (df[df.GoldenCross & (df.WeekChange > 3)].GoldenCross.count()/goldenCount)*100
        w1postiveFourPercentChange = (df[df.GoldenCross & (df.WeekChange > 4)].GoldenCross.count()/goldenCount)*100
        w1postiveFivePercentChange = (df[df.GoldenCross & (df.WeekChange > 5)].GoldenCross.count()/goldenCount)*100
        w1postiveTenPercentChange = (df[df.GoldenCross & (df.WeekChange > 10)].GoldenCross.count()/goldenCount)*100
        w1postiveFifteenPercentChange = (df[df.GoldenCross & (df.WeekChange > 15)].GoldenCross.count()/goldenCount)*100
        w1postiveTwentyPercentChange = (df[df.GoldenCross & (df.WeekChange > 20)].GoldenCross.count()/goldenCount)*100
        w1postiveThirtyPercentChange = (df[df.GoldenCross & (df.WeekChange > 30)].GoldenCross.count()/goldenCount)*100
        # Percentage of change after Death Cross week1
        w1negativeOnePercentChange = (df[df.DeathCross & (df.WeekChange < 1)].DeathCross.count()/deathCount)*100
        w1negativeTwoPercentChange = (df[df.DeathCross & (df.WeekChange < 2)].DeathCross.count()/deathCount)*100
        w1negativeThreePercentChange = (df[df.DeathCross & (df.WeekChange < 3)].DeathCross.count()/deathCount)*100
        w1negativeFourPercentChange = (df[df.DeathCross & (df.WeekChange < 4)].DeathCross.count()/deathCount)*100
        w1negativeFivePercentChange = (df[df.DeathCross & (df.WeekChange < 5)].DeathCross.count()/deathCount)*100
        w1negativeTenPercentChange = (df[df.DeathCross & (df.WeekChange < 10)].DeathCross.count()/deathCount)*100
        w1negativeFifteenPercentChange = (df[df.DeathCross & (df.WeekChange < 15)].DeathCross.count()/deathCount)*100
        w1negativeTwentyPercentChange = (df[df.DeathCross & (df.WeekChange < 20)].DeathCross.count()/deathCount)*100
        w1negativeThrityPercentChange = (df[df.DeathCross & (df.WeekChange < 30)].DeathCross.count()/deathCount)*100

        #     print("1%",postiveOnePercentChange)
    #     print("-1%",negativeOnePercentChange)

        # Percentage of change after Golden Cross week2
        w2postiveOnePercentChange = (df[df.GoldenCross & (df.TwoWeekChange > 1)].GoldenCross.count()/goldenCount)*100
        w2postiveTwoPercentChange = (df[df.GoldenCross & (df.TwoWeekChange > 2)].GoldenCross.count()/goldenCount)*100
        w2postiveThreePercentChange = (df[df.GoldenCross & (df.TwoWeekChange > 3)].GoldenCross.count()/goldenCount)*100
        w2postiveFourPercentChange = (df[df.GoldenCross & (df.TwoWeekChange > 4)].GoldenCross.count()/goldenCount)*100
        w2postiveFivePercentChange = (df[df.GoldenCross & (df.TwoWeekChange > 5)].GoldenCross.count()/goldenCount)*100
        w2postiveTenPercentChange = (df[df.GoldenCross & (df.TwoWeekChange > 10)].GoldenCross.count()/goldenCount)*100
        w2postiveFifteenPercentChange = (df[df.GoldenCross & (df.TwoWeekChange > 15)].GoldenCross.count()/goldenCount)*100
        w2postiveTwentyPercentChange = (df[df.GoldenCross & (df.TwoWeekChange > 20)].GoldenCross.count()/goldenCount)*100
        w2postiveThirtyPercentChange = (df[df.GoldenCross & (df.TwoWeekChange > 30)].GoldenCross.count()/goldenCount)*100

        # Percentage of change after Death Cross week2
        w2negativeOnePercentChange = (df[df.DeathCross & (df.TwoWeekChange < 1)].DeathCross.count()/deathCount)*100
        w2negativeTwoPercentChange = (df[df.DeathCross & (df.TwoWeekChange < 2)].DeathCross.count()/deathCount)*100
        w2negativeThreePercentChange = (df[df.DeathCross & (df.TwoWeekChange < 3)].DeathCross.count()/deathCount)*100
        w2negativeFourPercentChange = (df[df.DeathCross & (df.TwoWeekChange < 4)].DeathCross.count()/deathCount)*100
        w2negativeFivePercentChange = (df[df.DeathCross & (df.TwoWeekChange < 5)].DeathCross.count()/deathCount)*100
        w2negativeTenPercentChange = (df[df.DeathCross & (df.TwoWeekChange < 10)].DeathCross.count()/deathCount)*100
        w2negativeFifteenPercentChange = (df[df.DeathCross & (df.TwoWeekChange < 15)].DeathCross.count()/deathCount)*100
        w2negativeTwentyPercentChange = (df[df.DeathCross & (df.TwoWeekChange < 20)].DeathCross.count()/deathCount)*100
        w2negativeThirtyPercentChange = (df[df.DeathCross & (df.TwoWeekChange < 30)].DeathCross.count()/deathCount)*100

    #     print(postiveTwoPercentChange)
    #     print(negativeTwoPercentChange)

        # Percentage of change after Golden Cross week3
        w3postiveOnePercentChange = (df[df.GoldenCross & (df.ThreeWeekChange > 1)].GoldenCross.count()/goldenCount)*100
        w3postiveTwoPercentChange = (df[df.GoldenCross & (df.ThreeWeekChange > 2)].GoldenCross.count()/goldenCount)*100
        w3postiveThreePercentChange = (df[df.GoldenCross & (df.ThreeWeekChange > 3)].GoldenCross.count()/goldenCount)*100
        w3postiveFourPercentChange = (df[df.GoldenCross & (df.ThreeWeekChange > 4)].GoldenCross.count()/goldenCount)*100
        w3postiveFivePercentChange = (df[df.GoldenCross & (df.ThreeWeekChange > 5)].GoldenCross.count()/goldenCount)*100
        w3postiveTenPercentChange = (df[df.GoldenCross & (df.ThreeWeekChange > 10)].GoldenCross.count()/goldenCount)*100
        w3postiveFifteenPercentChange = (df[df.GoldenCross & (df.ThreeWeekChange > 15)].GoldenCross.count()/goldenCount)*100
        w3postiveTwentyPercentChange = (df[df.GoldenCross & (df.ThreeWeekChange > 20)].GoldenCross.count()/goldenCount)*100
        w3postiveThirtyPercentChange = (df[df.GoldenCross & (df.ThreeWeekChange > 30)].GoldenCross.count()/goldenCount)*100


        # Percentage of change after Death Cross week3
        w3negativeOnePercentChange = (df[df.DeathCross & (df.ThreeWeekChange < 1)].DeathCross.count()/deathCount)*100
        w3negativeTwoPercentChange = (df[df.DeathCross & (df.ThreeWeekChange < 2)].DeathCross.count()/deathCount)*100
        w3negativeThreePercentChange = (df[df.DeathCross & (df.ThreeWeekChange < 3)].DeathCross.count()/deathCount)*100
        w3negativeFourPercentChange = (df[df.DeathCross & (df.ThreeWeekChange < 4)].DeathCross.count()/deathCount)*100
        w3negativeFivePercentChange = (df[df.DeathCross & (df.ThreeWeekChange < 5)].DeathCross.count()/deathCount)*100
        w3negativeTenPercentChange = (df[df.DeathCross & (df.ThreeWeekChange < 10)].DeathCross.count()/deathCount)*100
        w3negativeFifteenPercentChange = (df[df.DeathCross & (df.ThreeWeekChange < 15)].DeathCross.count()/deathCount)*100
        w3negativeTwentyPercentChange = (df[df.DeathCross & (df.ThreeWeekChange < 20)].DeathCross.count()/deathCount)*100
        w3negativeThirtyPercentChange = (df[df.DeathCross & (df.ThreeWeekChange < 30)].DeathCross.count()/deathCount)*100

    #     print(postiveThreePercentChange)
    #     print(negativeThreePercentChange)

        # Percentage of change in Golden Cross in a month
        m1postiveOnePercentChange = (df[df.GoldenCross & (df.MonthChange > 1)].GoldenCross.count()/goldenCount)*100
        m1postiveTwoPercentChange = (df[df.GoldenCross & (df.MonthChange > 2)].GoldenCross.count()/goldenCount)*100
        m1postiveThreePercentChange = (df[df.GoldenCross & (df.MonthChange > 3)].GoldenCross.count()/goldenCount)*100
        m1postiveFourPercentChange = (df[df.GoldenCross & (df.MonthChange > 4)].GoldenCross.count()/goldenCount)*100
        m1postiveFivePercentChange = (df[df.GoldenCross & (df.MonthChange > 5)].GoldenCross.count()/goldenCount)*100
        m1postiveTenPercentChange = (df[df.GoldenCross & (df.MonthChange > 10)].GoldenCross.count()/goldenCount)*100
        m1postiveFifteenPercentChange = (df[df.GoldenCross & (df.MonthChange > 15)].GoldenCross.count()/goldenCount)*100
        m1postiveTwentyPercentChange = (df[df.GoldenCross & (df.MonthChange > 20)].GoldenCross.count()/goldenCount)*100
        m1postiveThirtyPercentChange = (df[df.GoldenCross & (df.MonthChange > 30)].GoldenCross.count()/goldenCount)*100

        # Percentage of change after Death Cross in a month
        m1negativeOnePercentChange = (df[df.DeathCross & (df.MonthChange < 1)].DeathCross.count()/deathCount)*100
        m1negativeTwoPercentChange = (df[df.DeathCross & (df.MonthChange < 2)].DeathCross.count()/deathCount)*100
        m1negativeThreePercentChange = (df[df.DeathCross & (df.MonthChange < 3)].DeathCross.count()/deathCount)*100
        m1negativeFourPercentChange = (df[df.DeathCross & (df.MonthChange < 4)].DeathCross.count()/deathCount)*100
        m1negativeFivePercentChange = (df[df.DeathCross & (df.MonthChange < 5)].DeathCross.count()/deathCount)*100
        m1negativeTenPercentChange = (df[df.DeathCross & (df.MonthChange < 10)].DeathCross.count()/deathCount)*100
        m1negativeFifteenPercentChange = (df[df.DeathCross & (df.MonthChange < 15)].DeathCross.count()/deathCount)*100
        m1negativeTwentyPercentChange = (df[df.DeathCross & (df.MonthChange < 20)].DeathCross.count()/deathCount)*100
        m1negativeThirtyPercentChange = (df[df.DeathCross & (df.MonthChange < 30)].DeathCross.count()/deathCount)*100

    #     print(postiveFourPercentChange)
    #     print(negativeFourPercentChange)

        # Percentage of change after Golden Cross in two months
        m2postiveOnePercentChange = (df[df.GoldenCross & (df.TwoMonthChange > 1)].GoldenCross.count()/goldenCount)*100
        m2postiveTwoPercentChange = (df[df.GoldenCross & (df.TwoMonthChange > 2)].GoldenCross.count()/goldenCount)*100
        m2postiveThreePercentChange = (df[df.GoldenCross & (df.TwoMonthChange > 3)].GoldenCross.count()/goldenCount)*100
        m2postiveFourPercentChange = (df[df.GoldenCross & (df.TwoMonthChange > 4)].GoldenCross.count()/goldenCount)*100
        m2postiveFivePercentChange = (df[df.GoldenCross & (df.TwoMonthChange > 5)].GoldenCross.count()/goldenCount)*100
        m2postiveTenPercentChange = (df[df.GoldenCross & (df.TwoMonthChange > 10)].GoldenCross.count()/goldenCount)*100
        m2postiveFifteenPercentChange = (df[df.GoldenCross & (df.TwoMonthChange > 15)].GoldenCross.count()/goldenCount)*100
        m2postiveTwentyPercentChange = (df[df.GoldenCross & (df.TwoMonthChange > 20)].GoldenCross.count()/goldenCount)*100
        m2postiveThirtyPercentChange = (df[df.GoldenCross & (df.TwoMonthChange > 30)].GoldenCross.count()/goldenCount)*100

        # Percentage of change after Death Cross in two months
        m2negativeOnePercentChange = (df[df.DeathCross & (df.TwoMonthChange < 1)].DeathCross.count()/deathCount)*100
        m2negativeTwoPercentChange = (df[df.DeathCross & (df.TwoMonthChange < 2)].DeathCross.count()/deathCount)*100
        m2negativeThreePercentChange = (df[df.DeathCross & (df.TwoMonthChange < 3)].DeathCross.count()/deathCount)*100
        m2negativeFourPercentChange = (df[df.DeathCross & (df.TwoMonthChange < 4)].DeathCross.count()/deathCount)*100
        m2negativeFivePercentChange = (df[df.DeathCross & (df.TwoMonthChange < 5)].DeathCross.count()/deathCount)*100
        m2negativeTenPercentChange = (df[df.DeathCross & (df.TwoMonthChange < 10)].DeathCross.count()/deathCount)*100
        m2negativeFifteenPercentChange = (df[df.DeathCross & (df.TwoMonthChange < 15)].DeathCross.count()/deathCount)*100
        m2negativeTwentyPercentChange = (df[df.DeathCross & (df.TwoMonthChange < 20)].DeathCross.count()/deathCount)*100
        m2negativeThirtyPercentChange = (df[df.DeathCross & (df.TwoMonthChange < 30)].DeathCross.count()/deathCount)*100

    #     print(postiveFivePercentChange)
    #     print(negativeFivePercentChange)

        # Percentage of change after Golden Cross in three months
        m3postiveOnePercentChange = (df[df.GoldenCross & (df.ThreeMonthChange > 1)].GoldenCross.count()/goldenCount)*100
        m3postiveTwoPercentChange = (df[df.GoldenCross & (df.ThreeMonthChange > 2)].GoldenCross.count()/goldenCount)*100
        m3postiveThreePercentChange = (df[df.GoldenCross & (df.ThreeMonthChange > 3)].GoldenCross.count()/goldenCount)*100
        m3postiveFourPercentChange = (df[df.GoldenCross & (df.ThreeMonthChange > 4)].GoldenCross.count()/goldenCount)*100
        m3postiveFivePercentChange = (df[df.GoldenCross & (df.ThreeMonthChange > 5)].GoldenCross.count()/goldenCount)*100
        m3postiveTenPercentChange = (df[df.GoldenCross & (df.ThreeMonthChange > 10)].GoldenCross.count()/goldenCount)*100
        m3postiveFifteenPercentChange = (df[df.GoldenCross & (df.ThreeMonthChange > 15)].GoldenCross.count()/goldenCount)*100
        m3postiveTwentyPercentChange = (df[df.GoldenCross & (df.ThreeMonthChange > 20)].GoldenCross.count()/goldenCount)*100
        m3postiveThirtyPercentChange = (df[df.GoldenCross & (df.ThreeMonthChange > 30)].GoldenCross.count()/goldenCount)*100


        # Percentage of change after Death Cross in three months
        m3negativeOnePercentChange = (df[df.DeathCross & (df.ThreeMonthChange < 1)].DeathCross.count()/deathCount)*100
        m3negativeTwoPercentChange = (df[df.DeathCross & (df.ThreeMonthChange < 2)].DeathCross.count()/deathCount)*100
        m3negativeThreePercentChange = (df[df.DeathCross & (df.ThreeMonthChange < 3)].DeathCross.count()/deathCount)*100
        m3negativeFourPercentChange = (df[df.DeathCross & (df.ThreeMonthChange < 4)].DeathCross.count()/deathCount)*100
        m3negativeFivePercentChange = (df[df.DeathCross & (df.ThreeMonthChange < 5)].DeathCross.count()/deathCount)*100
        m3negativeTenPercentChange = (df[df.DeathCross & (df.ThreeMonthChange < 10)].DeathCross.count()/deathCount)*100
        m3negativeFifteenPercentChange = (df[df.DeathCross & (df.ThreeMonthChange < 15)].DeathCross.count()/deathCount)*100
        m3negativeTwentyPercentChange = (df[df.DeathCross & (df.ThreeMonthChange < 20)].DeathCross.count()/deathCount)*100
        m3negativeThirtyPercentChange = (df[df.DeathCross & (df.ThreeMonthChange < 30)].DeathCross.count()/deathCount)*100



    #     print(postiveTenPercentChange)
    #     print(negativeTenPercentChange)

        # Percentage of change after Golden Cross in Four months
        m4postiveOnePercentChange = (df[df.GoldenCross & (df.FourMonthChange > 1)].GoldenCross.count()/goldenCount)*100
        m4postiveTwoPercentChange = (df[df.GoldenCross & (df.FourMonthChange > 2)].GoldenCross.count()/goldenCount)*100
        m4postiveThreePercentChange = (df[df.GoldenCross & (df.FourMonthChange > 3)].GoldenCross.count()/goldenCount)*100
        m4postiveFourPercentChange = (df[df.GoldenCross & (df.FourMonthChange > 4)].GoldenCross.count()/goldenCount)*100
        m4postiveFivePercentChange = (df[df.GoldenCross & (df.FourMonthChange > 5)].GoldenCross.count()/goldenCount)*100
        m4postiveTenPercentChange = (df[df.GoldenCross & (df.FourMonthChange > 10)].GoldenCross.count()/goldenCount)*100
        m4postiveFifteenPercentChange = (df[df.GoldenCross & (df.FourMonthChange > 15)].GoldenCross.count()/goldenCount)*100
        m4postiveTwentyPercentChange = (df[df.GoldenCross & (df.FourMonthChange > 20)].GoldenCross.count()/goldenCount)*100
        m4postiveThirtyPercentChange = (df[df.GoldenCross & (df.FourMonthChange > 30)].GoldenCross.count()/goldenCount)*100


        # Percentage of change after Death Cross in four months
        m4negativeOnePercentChange = (df[df.DeathCross & (df.FourMonthChange < 1)].DeathCross.count()/deathCount)*100
        m4negativeTwoPercentChange = (df[df.DeathCross & (df.FourMonthChange < 2)].DeathCross.count()/deathCount)*100
        m4negativeThreePercentChange = (df[df.DeathCross & (df.FourMonthChange < 3)].DeathCross.count()/deathCount)*100
        m4negativeFourPercentChange = (df[df.DeathCross & (df.FourMonthChange < 4)].DeathCross.count()/deathCount)*100
        m4negativeFivePercentChange = (df[df.DeathCross & (df.FourMonthChange < 5)].DeathCross.count()/deathCount)*100
        m4negativeTenPercentChange = (df[df.DeathCross & (df.FourMonthChange < 10)].DeathCross.count()/deathCount)*100
        m4negativeFifteenPercentChange = (df[df.DeathCross & (df.FourMonthChange < 15)].DeathCross.count()/deathCount)*100
        m4negativeTwentyPercentChange = (df[df.DeathCross & (df.FourMonthChange < 20)].DeathCross.count()/deathCount)*100
        m4negativeThirtyPercentChange = (df[df.DeathCross & (df.FourMonthChange < 30)].DeathCross.count()/deathCount)*100



        #     print(postiveFifPercentChange)
    #     print(negativeFifPercentChange)

        # Percentage of change after Golden Cross in five months
        m5postiveOnePercentChange = (df[df.GoldenCross & (df.FiveMonthChange > 1)].GoldenCross.count()/goldenCount)*100
        m5postiveTwoPercentChange = (df[df.GoldenCross & (df.FiveMonthChange > 2)].GoldenCross.count()/goldenCount)*100
        m5postiveThreePercentChange = (df[df.GoldenCross & (df.FiveMonthChange > 3)].GoldenCross.count()/goldenCount)*100
        m5postiveFourPercentChange = (df[df.GoldenCross & (df.FiveMonthChange > 4)].GoldenCross.count()/goldenCount)*100
        m5postiveFivePercentChange = (df[df.GoldenCross & (df.FiveMonthChange > 5)].GoldenCross.count()/goldenCount)*100
        m5postiveTenPercentChange = (df[df.GoldenCross & (df.FiveMonthChange > 10)].GoldenCross.count()/goldenCount)*100
        m5postiveFifteenPercentChange = (df[df.GoldenCross & (df.FiveMonthChange > 15)].GoldenCross.count()/goldenCount)*100
        m5postiveTwentyPercentChange = (df[df.GoldenCross & (df.FiveMonthChange > 20)].GoldenCross.count()/goldenCount)*100
        m5postiveThirtyPercentChange = (df[df.GoldenCross & (df.FiveMonthChange > 30)].GoldenCross.count()/goldenCount)*100

        # Percentage of change after Death Cross in Fivemonths months
        m5negativeOnePercentChange = (df[df.DeathCross & (df.FiveMonthChange < 1)].DeathCross.count()/deathCount)*100
        m5negativeTwoPercentChange = (df[df.DeathCross & (df.FiveMonthChange < 2)].DeathCross.count()/deathCount)*100
        m5negativeThreePercentChange = (df[df.DeathCross & (df.FiveMonthChange < 3)].DeathCross.count()/deathCount)*100
        m5negativeFourPercentChange = (df[df.DeathCross & (df.FiveMonthChange < 4)].DeathCross.count()/deathCount)*100
        m5negativeFivePercentChange = (df[df.DeathCross & (df.FiveMonthChange < 5)].DeathCross.count()/deathCount)*100
        m5negativeTenPercentChange = (df[df.DeathCross & (df.FiveMonthChange < 10)].DeathCross.count()/deathCount)*100
        m5negativeFifteenPercentChange = (df[df.DeathCross & (df.FiveMonthChange < 15)].DeathCross.count()/deathCount)*100
        m5negativeTwentyPercentChange = (df[df.DeathCross & (df.FiveMonthChange < 20)].DeathCross.count()/deathCount)*100
        m5negativeThirtyPercentChange = (df[df.DeathCross & (df.FiveMonthChange < 30)].DeathCross.count()/deathCount)*100



        #     print(postiveTwenPercentChange)
    #     print(negativeTwenPercentChange)
        m6postiveOnePercentChange = (df[df.GoldenCross & (df.SixMonthChange > 1)].GoldenCross.count()/goldenCount)*100
        m6postiveTwoPercentChange = (df[df.GoldenCross & (df.SixMonthChange > 2)].GoldenCross.count()/goldenCount)*100
        m6postiveThreePercentChange = (df[df.GoldenCross & (df.SixMonthChange > 3)].GoldenCross.count()/goldenCount)*100
        m6postiveFourPercentChange = (df[df.GoldenCross & (df.SixMonthChange > 4)].GoldenCross.count()/goldenCount)*100
        m6postiveFivePercentChange = (df[df.GoldenCross & (df.SixMonthChange > 5)].GoldenCross.count()/goldenCount)*100
        m6postiveTenPercentChange = (df[df.GoldenCross & (df.SixMonthChange > 10)].GoldenCross.count()/goldenCount)*100
        m6postiveFifteenPercentChange = (df[df.GoldenCross & (df.SixMonthChange > 15)].GoldenCross.count()/goldenCount)*100
        m6postiveTwentyPercentChange = (df[df.GoldenCross & (df.SixMonthChange > 20)].GoldenCross.count()/goldenCount)*100
        m6postiveThirtyPercentChange = (df[df.GoldenCross & (df.SixMonthChange > 30)].GoldenCross.count()/goldenCount)*100

        m6negativeOnePercentChange = (df[df.DeathCross & (df.SixMonthChange < 1)].DeathCross.count()/deathCount)*100
        m6negativeTwoPercentChange = (df[df.DeathCross & (df.SixMonthChange < 2)].DeathCross.count()/deathCount)*100
        m6negativeThreePercentChange = (df[df.DeathCross & (df.SixMonthChange < 3)].DeathCross.count()/deathCount)*100
        m6negativeFourPercentChange = (df[df.DeathCross & (df.SixMonthChange < 4)].DeathCross.count()/deathCount)*100
        m6negativeFivePercentChange = (df[df.DeathCross & (df.SixMonthChange < 5)].DeathCross.count()/deathCount)*100
        m6negativeTenPercentChange = (df[df.DeathCross & (df.SixMonthChange < 10)].DeathCross.count()/deathCount)*100
        m6negativeFifteenPercentChange = (df[df.DeathCross & (df.SixMonthChange < 15)].DeathCross.count()/deathCount)*100
        m6negativeTwentyPercentChange = (df[df.DeathCross & (df.SixMonthChange < 20)].DeathCross.count()/deathCount)*100
        m6negativeThirtyPercentChange = (df[df.DeathCross & (df.SixMonthChange < 30)].DeathCross.count()/deathCount)*100


        Analysis = pd.DataFrame(index=index, columns=columns)
        DeathAnalysis = pd.DataFrame(index=index, columns=columns)
        #Week1 Row all percentages after Golden cross
        Analysis.loc[Analysis.index[0], '1%'] = w1postiveOnePercentChange
        Analysis.loc[Analysis.index[0], '2%'] = w1postiveTwoPercentChange
        Analysis.loc[Analysis.index[0], '3%'] = w1postiveThreePercentChange
        Analysis.loc[Analysis.index[0], '4%'] = w1postiveFourPercentChange
        Analysis.loc[Analysis.index[0], '5%'] = w1postiveFivePercentChange
        Analysis.loc[Analysis.index[0], '10%'] = w1postiveTenPercentChange
        Analysis.loc[Analysis.index[0], '15%'] = w1postiveFifteenPercentChange
        Analysis.loc[Analysis.index[0], '20%'] = w1postiveTwentyPercentChange
        Analysis.loc[Analysis.index[0], '30%'] = w1postiveThirtyPercentChange

        #Week2 Row all percentages after Golden cross
        Analysis.loc[Analysis.index[1], '1%'] = w2postiveOnePercentChange
        Analysis.loc[Analysis.index[1], '2%'] = w2postiveTwoPercentChange
        Analysis.loc[Analysis.index[1], '3%'] = w2postiveThreePercentChange
        Analysis.loc[Analysis.index[1], '4%'] = w2postiveFourPercentChange
        Analysis.loc[Analysis.index[1], '5%'] = w2postiveFivePercentChange
        Analysis.loc[Analysis.index[1], '10%'] = w2postiveTenPercentChange
        Analysis.loc[Analysis.index[1], '15%'] = w2postiveFifteenPercentChange
        Analysis.loc[Analysis.index[1], '20%'] = w2postiveTwentyPercentChange
        Analysis.loc[Analysis.index[1], '30%'] = w2postiveThirtyPercentChange

        #Week3 Row all percentages after Golden cross
        Analysis.loc[Analysis.index[2], '1%'] = w3postiveOnePercentChange
        Analysis.loc[Analysis.index[2], '2%'] = w3postiveTwoPercentChange
        Analysis.loc[Analysis.index[2], '3%'] = w3postiveThreePercentChange
        Analysis.loc[Analysis.index[2], '4%'] = w3postiveFourPercentChange
        Analysis.loc[Analysis.index[2], '5%'] = w3postiveFivePercentChange
        Analysis.loc[Analysis.index[2], '10%'] = w3postiveTenPercentChange
        Analysis.loc[Analysis.index[2], '15%'] = w3postiveFifteenPercentChange
        Analysis.loc[Analysis.index[2], '20%'] = w3postiveTwentyPercentChange
        Analysis.loc[Analysis.index[2], '30%'] = w3postiveThirtyPercentChange

        #Month Row all percentages after Golden cross
        Analysis.loc[Analysis.index[3], '1%'] = m1postiveOnePercentChange
        Analysis.loc[Analysis.index[3], '2%'] = m1postiveTwoPercentChange
        Analysis.loc[Analysis.index[3], '3%'] = m1postiveThreePercentChange
        Analysis.loc[Analysis.index[3], '4%'] = m1postiveFourPercentChange
        Analysis.loc[Analysis.index[3], '5%'] = m1postiveFivePercentChange
        Analysis.loc[Analysis.index[3], '10%'] = m1postiveTenPercentChange
        Analysis.loc[Analysis.index[3], '15%'] = m1postiveFifteenPercentChange
        Analysis.loc[Analysis.index[3], '20%'] = m1postiveTwentyPercentChange
        Analysis.loc[Analysis.index[3], '30%'] = m1postiveThirtyPercentChange

        #Twomonths Row all percentages after Golden cross
        Analysis.loc[Analysis.index[4], '1%'] = m2postiveOnePercentChange
        Analysis.loc[Analysis.index[4], '2%'] = m2postiveTwoPercentChange
        Analysis.loc[Analysis.index[4], '3%'] = m2postiveThreePercentChange
        Analysis.loc[Analysis.index[4], '4%'] = m2postiveFourPercentChange
        Analysis.loc[Analysis.index[4], '5%'] = m2postiveFivePercentChange
        Analysis.loc[Analysis.index[4], '10%'] = m2postiveTenPercentChange
        Analysis.loc[Analysis.index[4], '15%'] = m2postiveFifteenPercentChange
        Analysis.loc[Analysis.index[4], '20%'] = m2postiveTwentyPercentChange
        Analysis.loc[Analysis.index[4], '30%'] = m2postiveThirtyPercentChange
        #Threemonths Row all percentages after Golden cross
        Analysis.loc[Analysis.index[5], '1%'] = m3postiveOnePercentChange
        Analysis.loc[Analysis.index[5], '2%'] = m3postiveTwoPercentChange
        Analysis.loc[Analysis.index[5], '3%'] = m3postiveThreePercentChange
        Analysis.loc[Analysis.index[5], '4%'] = m3postiveFourPercentChange
        Analysis.loc[Analysis.index[5], '5%'] = m3postiveFivePercentChange
        Analysis.loc[Analysis.index[5], '10%'] = m3postiveTenPercentChange
        Analysis.loc[Analysis.index[5], '15%'] = m3postiveFifteenPercentChange
        Analysis.loc[Analysis.index[5], '20%'] = m3postiveTwentyPercentChange
        Analysis.loc[Analysis.index[5], '30%'] = m3postiveThirtyPercentChange

        #Fourmonths Row all percentages after Golden cross
        Analysis.loc[Analysis.index[6], '1%'] = m4postiveOnePercentChange
        Analysis.loc[Analysis.index[6], '2%'] = m4postiveTwoPercentChange
        Analysis.loc[Analysis.index[6], '3%'] = m4postiveThreePercentChange
        Analysis.loc[Analysis.index[6], '4%'] = m4postiveFourPercentChange
        Analysis.loc[Analysis.index[6], '5%'] = m4postiveFivePercentChange
        Analysis.loc[Analysis.index[6], '10%'] = m4postiveTenPercentChange
        Analysis.loc[Analysis.index[6], '15%'] = m4postiveFifteenPercentChange
        Analysis.loc[Analysis.index[6], '20%'] = m4postiveTwentyPercentChange
        Analysis.loc[Analysis.index[6], '30%'] = m4postiveThirtyPercentChange


        #Fivemonths Row all percentages after Golden cross
        Analysis.loc[Analysis.index[7], '1%'] = m5postiveOnePercentChange
        Analysis.loc[Analysis.index[7], '2%'] = m5postiveTwoPercentChange
        Analysis.loc[Analysis.index[7], '3%'] = m5postiveThreePercentChange
        Analysis.loc[Analysis.index[7], '4%'] = m5postiveFourPercentChange
        Analysis.loc[Analysis.index[7], '5%'] = m5postiveFivePercentChange
        Analysis.loc[Analysis.index[7], '10%'] = m5postiveTenPercentChange
        Analysis.loc[Analysis.index[7], '15%'] = m5postiveFifteenPercentChange
        Analysis.loc[Analysis.index[7], '20%'] = m5postiveTwentyPercentChange
        Analysis.loc[Analysis.index[7], '30%'] = m5postiveThirtyPercentChange


        #sixmonths Row all percentages after Golden cross
        Analysis.loc[Analysis.index[8], '1%'] = m6postiveOnePercentChange
        Analysis.loc[Analysis.index[8], '2%'] = m6postiveTwoPercentChange
        Analysis.loc[Analysis.index[8], '3%'] = m6postiveThreePercentChange
        Analysis.loc[Analysis.index[8], '4%'] = m6postiveFourPercentChange
        Analysis.loc[Analysis.index[8], '5%'] = m6postiveFivePercentChange
        Analysis.loc[Analysis.index[8], '10%'] = m6postiveTenPercentChange
        Analysis.loc[Analysis.index[8], '15%'] = m6postiveFifteenPercentChange
        Analysis.loc[Analysis.index[8], '20%'] = m6postiveTwentyPercentChange
        Analysis.loc[Analysis.index[8], '30%'] = m6postiveThirtyPercentChange


        print(Analysis)

        #Week1 Row all percentages after Death cross
        DeathAnalysis.loc[DeathAnalysis.index[0], '1%'] = w1negativeOnePercentChange
        DeathAnalysis.loc[DeathAnalysis.index[0], '2%'] = w1negativeTwoPercentChange
        DeathAnalysis.loc[DeathAnalysis.index[0], '3%'] = w1negativeThreePercentChange
        DeathAnalysis.loc[DeathAnalysis.index[0], '4%'] = w1negativeFourPercentChange
        DeathAnalysis.loc[DeathAnalysis.index[0], '5%'] = w1negativeFivePercentChange
        DeathAnalysis.loc[DeathAnalysis.index[0], '10%'] = w1negativeTenPercentChange
        DeathAnalysis.loc[DeathAnalysis.index[0], '15%'] = w1negativeFifteenPercentChange
        DeathAnalysis.loc[DeathAnalysis.index[0], '20%'] = w1negativeTwentyPercentChange
        DeathAnalysis.loc[DeathAnalysis.index[0], '30%'] = w1negativeThrityPercentChange
        #Week2 Row all percentages after Death cross
        DeathAnalysis.loc[DeathAnalysis.index[1], '1%'] = w2negativeOnePercentChange
        DeathAnalysis.loc[DeathAnalysis.index[1], '2%'] = w2negativeTwoPercentChange
        DeathAnalysis.loc[DeathAnalysis.index[1], '3%'] = w2negativeThreePercentChange
        DeathAnalysis.loc[DeathAnalysis.index[1], '4%'] = w2negativeFourPercentChange
        DeathAnalysis.loc[DeathAnalysis.index[1], '5%'] = w2negativeFivePercentChange
        DeathAnalysis.loc[DeathAnalysis.index[1], '10%'] = w2negativeTenPercentChange
        DeathAnalysis.loc[DeathAnalysis.index[1], '15%'] = w2negativeFifteenPercentChange
        DeathAnalysis.loc[DeathAnalysis.index[1], '20%'] = w2negativeTwentyPercentChange
        DeathAnalysis.loc[DeathAnalysis.index[1], '30%'] = w2negativeThirtyPercentChange
        #Week3 Row all percentages after Death cross
        DeathAnalysis.loc[DeathAnalysis.index[2], '1%'] = w3negativeOnePercentChange
        DeathAnalysis.loc[DeathAnalysis.index[2], '2%'] = w3negativeTwoPercentChange
        DeathAnalysis.loc[DeathAnalysis.index[2], '3%'] = w3negativeThreePercentChange
        DeathAnalysis.loc[DeathAnalysis.index[2], '4%'] = w3negativeFourPercentChange
        DeathAnalysis.loc[DeathAnalysis.index[2], '5%'] = w3negativeFivePercentChange
        DeathAnalysis.loc[DeathAnalysis.index[2], '10%'] = w3negativeTenPercentChange
        DeathAnalysis.loc[DeathAnalysis.index[2], '15%'] = w3negativeFifteenPercentChange
        DeathAnalysis.loc[DeathAnalysis.index[2], '20%'] = w3negativeTwentyPercentChange
        DeathAnalysis.loc[DeathAnalysis.index[2], '30%'] = w3negativeThirtyPercentChange

        #month Row all percentages after Death cross
        DeathAnalysis.loc[DeathAnalysis.index[3], '1%'] = m1negativeOnePercentChange
        DeathAnalysis.loc[DeathAnalysis.index[3], '2%'] = m1negativeTwoPercentChange
        DeathAnalysis.loc[DeathAnalysis.index[3], '3%'] = m1negativeThreePercentChange
        DeathAnalysis.loc[DeathAnalysis.index[3], '4%'] = m1negativeFourPercentChange
        DeathAnalysis.loc[DeathAnalysis.index[3], '5%'] = m1negativeFivePercentChange
        DeathAnalysis.loc[DeathAnalysis.index[3], '10%'] = m1negativeTenPercentChange
        DeathAnalysis.loc[DeathAnalysis.index[3], '15%'] = m1negativeFifteenPercentChange
        DeathAnalysis.loc[DeathAnalysis.index[3], '20%'] = m1negativeTwentyPercentChange
        DeathAnalysis.loc[DeathAnalysis.index[3], '30%'] = m1negativeThirtyPercentChange
        #Twomonths Row all percentages after Death cross
        DeathAnalysis.loc[DeathAnalysis.index[4], '1%'] = m2negativeOnePercentChange
        DeathAnalysis.loc[DeathAnalysis.index[4], '2%'] = m2negativeTwoPercentChange
        DeathAnalysis.loc[DeathAnalysis.index[4], '3%'] = m2negativeThreePercentChange
        DeathAnalysis.loc[DeathAnalysis.index[4], '4%'] = m2negativeFourPercentChange
        DeathAnalysis.loc[DeathAnalysis.index[4], '5%'] = m2negativeFivePercentChange
        DeathAnalysis.loc[DeathAnalysis.index[4], '10%'] = m2negativeTenPercentChange
        DeathAnalysis.loc[DeathAnalysis.index[4], '15%'] = m2negativeFifteenPercentChange
        DeathAnalysis.loc[DeathAnalysis.index[4], '20%'] = m2negativeTwentyPercentChange
        DeathAnalysis.loc[DeathAnalysis.index[4], '30%'] = m2negativeThirtyPercentChange
        #Threeomonths Row all percentages after Death cross
        DeathAnalysis.loc[DeathAnalysis.index[5], '1%'] = m3negativeOnePercentChange
        DeathAnalysis.loc[DeathAnalysis.index[5], '2%'] = m3negativeTwoPercentChange
        DeathAnalysis.loc[DeathAnalysis.index[5], '3%'] = m3negativeThreePercentChange
        DeathAnalysis.loc[DeathAnalysis.index[5], '4%'] = m3negativeFourPercentChange
        DeathAnalysis.loc[DeathAnalysis.index[5], '5%'] = m3negativeFivePercentChange
        DeathAnalysis.loc[DeathAnalysis.index[5], '10%'] = m3negativeTenPercentChange
        DeathAnalysis.loc[DeathAnalysis.index[5], '15%'] = m3negativeFifteenPercentChange
        DeathAnalysis.loc[DeathAnalysis.index[5], '20%'] = m3negativeTwentyPercentChange
        DeathAnalysis.loc[DeathAnalysis.index[5], '30%'] = m3negativeThirtyPercentChange
        #Fourmonths Row all percentages after Death cross
        DeathAnalysis.loc[DeathAnalysis.index[6], '1%'] = m4negativeOnePercentChange
        DeathAnalysis.loc[DeathAnalysis.index[6], '2%'] = m4negativeTwoPercentChange
        DeathAnalysis.loc[DeathAnalysis.index[6], '3%'] = m4negativeThreePercentChange
        DeathAnalysis.loc[DeathAnalysis.index[6], '4%'] = m4negativeFourPercentChange
        DeathAnalysis.loc[DeathAnalysis.index[6], '5%'] = m4negativeFivePercentChange
        DeathAnalysis.loc[DeathAnalysis.index[6], '10%'] = m4negativeTenPercentChange
        DeathAnalysis.loc[DeathAnalysis.index[6], '15%'] = m4negativeFifteenPercentChange
        DeathAnalysis.loc[DeathAnalysis.index[6], '20%'] = m4negativeTwentyPercentChange
        DeathAnalysis.loc[DeathAnalysis.index[6], '30%'] = m4negativeThirtyPercentChange

        #Fivemonths Row all percentages after Death cross
        DeathAnalysis.loc[DeathAnalysis.index[7], '1%'] = m5negativeOnePercentChange
        DeathAnalysis.loc[DeathAnalysis.index[7], '2%'] = m5negativeTwoPercentChange
        DeathAnalysis.loc[DeathAnalysis.index[7], '3%'] = m5negativeThreePercentChange
        DeathAnalysis.loc[DeathAnalysis.index[7], '4%'] = m5negativeFourPercentChange
        DeathAnalysis.loc[DeathAnalysis.index[7], '5%'] = m5negativeFivePercentChange
        DeathAnalysis.loc[DeathAnalysis.index[7], '10%'] = m5negativeTenPercentChange
        DeathAnalysis.loc[DeathAnalysis.index[7], '15%'] = m5negativeFifteenPercentChange
        DeathAnalysis.loc[DeathAnalysis.index[7], '20%'] = m5negativeTwentyPercentChange
        DeathAnalysis.loc[DeathAnalysis.index[7], '30%'] = m5negativeThirtyPercentChange

        #Sixmonths Row all percentages after Death cross
        DeathAnalysis.loc[DeathAnalysis.index[8], '1%'] = m6negativeOnePercentChange
        DeathAnalysis.loc[DeathAnalysis.index[8], '2%'] = m6negativeTwoPercentChange
        DeathAnalysis.loc[DeathAnalysis.index[8], '3%'] = m6negativeThreePercentChange
        DeathAnalysis.loc[DeathAnalysis.index[8], '4%'] = m6negativeFourPercentChange
        DeathAnalysis.loc[DeathAnalysis.index[8], '5%'] = m6negativeFivePercentChange
        DeathAnalysis.loc[DeathAnalysis.index[8], '10%'] = m6negativeTenPercentChange
        DeathAnalysis.loc[DeathAnalysis.index[8], '15%'] = m6negativeFifteenPercentChange
        DeathAnalysis.loc[DeathAnalysis.index[8], '20%'] = m6negativeTwentyPercentChange
        DeathAnalysis.loc[DeathAnalysis.index[8], '30%'] = m6negativeThirtyPercentChange
        print(DeathAnalysis)
Analysis.to_csv (r'C:\Users\Mudabbir\Desktop\Winter Classes\813\Case3 Screenshots\Goldencross.csv', index = True, header=True) 
DeathAnalysis.to_csv (r'C:\Users\Mudabbir\Desktop\Winter Classes\813\Case3 Screenshots\Deathcross.csv', index = True, header=True) 


In [60]:

Analysis.loc[Analysis.index[0], '1%'] = postiveOnePercentChange

Analysis.loc[Analysis.index[1], '2%'] = postiveTwoPercentChange

Analysis.loc[Analysis.index[2], '3%'] = postiveThreePercentChange

Analysis.loc[Analysis.index[3], '4%'] = postiveFourPercentChange

Analysis.loc[Analysis.index[4], '5%'] = postiveFivePercentChange

Analysis.loc[Analysis.index[5], '10%'] = postiveTenPercentChange

Analysis.loc[Analysis.index[6], '15%'] = postiveFifPercentChange

Analysis.loc[Analysis.index[7], '20%'] = postiveTwenPercentChange

Analysis.loc[Analysis.index[8], '30%'] = 0

Analysis

,1%,2%,3%,4%,5%,10%,15%,20%,30%
Week1,50.8475,35.2941,17.6471,0,0,0,0,0,0
Week2,47.0588,47.0588,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Week3,NaN,NaN,23.5294,NaN,NaN,NaN,NaN,NaN,NaN
Month,NaN,NaN,NaN,70.5882,NaN,NaN,NaN,NaN,NaN
2Month,NaN,NaN,NaN,NaN,64.7059,NaN,NaN,NaN,NaN
3Month,NaN,NaN,NaN,NaN,NaN,41.1765,NaN,NaN,NaN
4Month,NaN,NaN,NaN,NaN,NaN,NaN,11.7647,NaN,NaN
5Month,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5.88235,NaN
6Month,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
